In [6]:
import yfinance as yf
import pandas as pd
import sqlite3
import os
import pandas as pd
import pandas_ta as ta
from sqlalchemy import create_engine


# --- 1. ÁREA DE CONFIGURAÇÃO (FÁCIL DE EDITAR) ---

TICKERS = [
    'ABEV3.SA', 'ALPK3.SA', 'AMAR3.SA', 'AMBP3.SA', 'ANIM3.SA',
    'ASAI3.SA', 'AURE3.SA', 'AZEV3.SA', 'AZTE3.SA', 'AZUL4.SA',
    'AZZA3.SA', 'B3SA3.SA', 'BBAS3.SA', 'BBSE3.SA', 'BEEF3.SA',
    'BIOM3.SA', 'BLAU3.SA', 'BMGB4.SA', 'BPAC11.SA', 'BPAN4.SA',
    'BRAP4.SA', 'BRAV3.SA', 'BRFS3.SA', 'BRKM5.SA', 'BRST3.SA',
    'CAML3.SA', 'CASH3.SA', 'CBAV3.SA', 'CMIG4.SA', 'CMIN3.SA',
    'COGN3.SA', 'CPLE6.SA', 'CSAN3.SA', 'CSED3.SA', 'CSNA3.SA',
    'CVCB3.SA', 'CYRE3.SA', 'DASA3.SA', 'DXCO3.SA', 'ECOR3.SA',
    'ELET3.SA', 'EMBR3.SA', 'ENEV3.SA', 'ENJU3.SA',
    'EQTL3.SA', 'EVEN3.SA', 'EZTC3.SA', 'FIQE3.SA', 'FLRY3.SA',
    'GGBR4.SA', 'GFSA3.SA', 'GMAT3.SA', 'GOAU4.SA', 'HAPV3.SA',
    'HBOR3.SA', 'HBRE3.SA', 'HBSA3.SA', 'HYPE3.SA', 'INBR32.SA',
    'IRBR3.SA', 'ITSA4.SA', 'ITUB4.SA', 'JHSF3.SA', 'JSLG3.SA',
    'KLBN11.SA', 'LIGT3.SA', 'LREN3.SA', 'LWSA3.SA', 'MBRF3.SA',
    'MEAL3.SA', 'MELK3.SA', 'MGLU3.SA', 'MOTV3.SA', 'MRVE3.SA',
    'MTRE3.SA', 'NATU3.SA', 'NEOE3.SA', 'ONCO3.SA', 'PCAR3.SA',
    'PETR4.SA', 'PETZ3.SA', 'PFRM3.SA', 'PGMN3.SA', 'PRIO3.SA',
    'PSSA3.SA', 'QUAL3.SA', 'RADL3.SA', 'RAIL3.SA', 'RAIZ4.SA',
    'RDOR3.SA', 'RENT3.SA', 'ROMI3.SA', 'SANB11.SA', 'SBSP3.SA',
    'SLCE3.SA', 'SUZB3.SA', 'TIMS3.SA', 'TOTS3.SA',
    'TUPY3.SA', 'UGPA3.SA', 'USIM5.SA', 'VALE3.SA', 'VAMO3.SA',
    'VBBR3.SA', 'VIVA3.SA', 'VIVT3.SA', 'WEGE3.SA', 'YDUQ3.SA'
]

DATABASE_FILE = 'dados_acoes.db'
PERIODO_DADOS = 'max'

# --- FIM DA CONFIGURAÇÃO ---


# 2. Dicionário dos intervalos
intervalos = {
    'candles_diarios': '1d',
    'candles_semanais': '1wk',
    'candles_mensais': '1mo'
}

# 3. Conexão com o banco de dados
print(f"Iniciando conexão com o banco de dados '{DATABASE_FILE}'...")
conn = sqlite3.connect(DATABASE_FILE)

try:
    # 4. Loop principal para baixar os dados
    for tabela_sql, intervalo_yf in intervalos.items():
        print(f"\nBuscando dados para: {tabela_sql} (Intervalo: {intervalo_yf})")
        
        lista_dfs_por_ticker = []
        
        for ticker in TICKERS:
            print(f"  Baixando {ticker}...")
            
            # Baixa os dados usando yfinance
            data = yf.download(ticker, period=PERIODO_DADOS, interval=intervalo_yf)
            
            # --- INÍCIO DA CORREÇÃO ---
            # Resolve o problema de MultiIndex (se houver)
            if isinstance(data.columns, pd.MultiIndex):
                data.columns = data.columns.get_level_values(0)
            # --- FIM DA CORREÇÃO ---
            
            if data.empty:
                print(f"  Sem dados para {ticker} no intervalo {intervalo_yf}.")
                continue
                
            # Adiciona o nome do ticker como uma coluna
            data['Ticker'] = ticker
            
            # Transforma o índice (Date) em uma coluna
            data.reset_index(inplace=True)
            
            # Renomeia a coluna de data para ser mais clara
            if 'Date' in data.columns:
                data.rename(columns={'Date': 'Datetime'}, inplace=True)
            
            lista_dfs_por_ticker.append(data)
            
        # 5. Combina todos os dataframes
        if not lista_dfs_por_ticker:
            print(f"Nenhum dado baixado para o intervalo {intervalo_yf}.")
            continue
            
        df_completo = pd.concat(lista_dfs_por_ticker, ignore_index=True)
        
        # 6. Limpa e padroniza os nomes das colunas para SQL
        # Esta linha agora vai funcionar!
        df_completo.columns = df_completo.columns.str.lower().str.replace(' ', '_')
        
        # 7. Salva o dataframe combinado na tabela SQL
        print(f"  Salvando dados na tabela SQL: '{tabela_sql}'...")
        df_completo.to_sql(
            tabela_sql, 
            conn, 
            if_exists='replace',
            index=False
        )
        
        print(f"  Tabela '{tabela_sql}' salva com sucesso!")

    print(f"\nProcesso concluído! Todos os dados foram salvos em '{DATABASE_FILE}'.")

except Exception as e:
    print(f"\nOcorreu um erro: {e}")

finally:
    # 8. Fecha a conexão com o banco de dados
    conn.close()
    print("Conexão com o banco de dados fechada.")

Iniciando conexão com o banco de dados 'dados_acoes.db'...

Buscando dados para: candles_diarios (Intervalo: 1d)
  Baixando ABEV3.SA...


C:\Users\Gustavo\AppData\Local\Temp\ipykernel_4500\3608632239.py:65: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, period=PERIODO_DADOS, interval=intervalo_yf)
[*********************100%***********************]  1 of 1 completed
C:\Users\Gustavo\AppData\Local\Temp\ipykernel_4500\3608632239.py:65: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, period=PERIODO_DADOS, interval=intervalo_yf)


  Baixando ALPK3.SA...


[*********************100%***********************]  1 of 1 completed
C:\Users\Gustavo\AppData\Local\Temp\ipykernel_4500\3608632239.py:65: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, period=PERIODO_DADOS, interval=intervalo_yf)


  Baixando AMAR3.SA...


...

  Baixando WEGE3.SA...


[*********************100%***********************]  1 of 1 completed
C:\Users\Gustavo\AppData\Local\Temp\ipykernel_4500\3608632239.py:65: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, period=PERIODO_DADOS, interval=intervalo_yf)


  Baixando YDUQ3.SA...


[*********************100%***********************]  1 of 1 completed


  Salvando dados na tabela SQL: 'candles_mensais'...
  Tabela 'candles_mensais' salva com sucesso!

Processo concluído! Todos os dados foram salvos em 'dados_acoes.db'.
Conexão com o banco de dados fechada.


In [7]:
# Define o nome do arquivo do banco de dados
db_file = "dados_acoes.db"

# Cria a "engine" de conexão do SQLAlchemy
# O prefixo 'sqlite:///' informa que é um arquivo SQLite
engine = create_engine(f'sqlite:///{db_file}')

try:
    # Carrega cada tabela em um DataFrame do Pandas
    # O Pandas usará a 'engine' para ler os dados
    candles_diarios = pd.read_sql("candles_diarios", engine)
    candles_semanais = pd.read_sql("candles_semanais", engine)
    candles_mensais = pd.read_sql("candles_mensais", engine)

    print("Tabelas carregadas com sucesso do banco de dados!")
    print(f"Total de registros 'candles_diarios': {len(candles_diarios)}")
    print(f"Total de registros 'candles_semanais': {len(candles_semanais)}")
    print(f"Total de registros 'candles_mensais': {len(candles_mensais)}")

except Exception as e:
    print(f"Erro ao ler o banco de dados: {e}")
    print("Verifique se o arquivo 'dados_acoes.db' está na mesma pasta do seu Jupyter Notebook.")


# --- AJUSTE ESTE MAPA CUIDADOSAMENTE ---
mapa_colunas = {
    'Data': 'datetime',      
    'Abertura': 'open',  
    'Maxima': 'high',    
    'Minima': 'low',     
    'Fechamento': 'close', 
    'Volume': 'volume'   
}

try:
    # Renomeia colunas, criando NOVOS dataframes "processados"
    diarios_proc = candles_diarios.rename(columns=mapa_colunas)
    semanais_proc = candles_semanais.rename(columns=mapa_colunas)
    mensais_proc = candles_mensais.rename(columns=mapa_colunas)

    print("DataFrames processados (clones) foram criados após renomeação.")
    
    # Ordena os dataframes processados (isso também cria cópias)
    if 'date' in diarios_proc.columns:
        diarios_proc = diarios_proc.sort_values(by='date')
        semanais_proc = semanais_proc.sort_values(by='date')
        mensais_proc = mensais_proc.sort_values(by='date')
        print("DataFrames processados foram ordenados por data.")

    print("\n--- Verificação ---")
    print(f"Colunas Originais (ex: diarios): {candles_diarios.columns.to_list()}")
    print(f"Colunas Processadas (ex: diarios_proc): {diarios_proc.columns.to_list()}")

except Exception as e:
    print(f"Erro ao renomear colunas: {e}")
    print("Verifique se os nomes no 'mapa_colunas' (lado esquerdo) estão corretos.")

Tabelas carregadas com sucesso do banco de dados!
Total de registros 'candles_diarios': 411037
Total de registros 'candles_semanais': 85971
Total de registros 'candles_mensais': 19802
DataFrames processados (clones) foram criados após renomeação.

--- Verificação ---
Colunas Originais (ex: diarios): ['datetime', 'close', 'high', 'low', 'open', 'volume', 'ticker']
Colunas Processadas (ex: diarios_proc): ['datetime', 'close', 'high', 'low', 'open', 'volume', 'ticker']


In [8]:
def calcular_indicadores_tecnicos(df_original):
    """
    Calcula um conjunto de indicadores técnicos usando pandas_ta 
    e os anexa a uma cópia do DataFrame.
    """
    # Cria uma cópia para não alterar o DataFrame de entrada
    df = df_original.copy()
    
    # 1. Média Móvel Exponencial (EMA) de 9 períodos
    df.ta.ema(length=9, append=True)
    
    # 2. Média Móvel Simples (SMA) de 21 períodos
    df.ta.sma(length=21, append=True)
    
    # 3. Média Móvel Simples (SMA) de 50 períodos
    df.ta.sma(length=50, append=True)
    
    # 4. Média Móvel Simples (SMA) de 200 períodos
    df.ta.sma(length=200, append=True)
    
    # 5. RSI (Índice de Força Relativa)
    df.ta.rsi(length=14, append=True)
    
    # 6. MACD
    df.ta.macd(append=True)
    
    # 7. Bandas de Bollinger (BBANDS)
    df.ta.bbands(length=20, append=True)
    
    # 8. Oscilador Estocástico (STOCH)
    df.ta.stoch(append=True)
    
    # 9. OBV (On-Balance Volume)
    df.ta.obv(append=True)
    
    # 10. ATR (Average True Range)
    df.ta.atr(length=14, append=True)
    
    # 11. ADX (Average Directional Index)
    df.ta.adx(length=14, append=True)
    
    return df

print("Função 'calcular_indicadores_tecnicos' definida.")

Função 'calcular_indicadores_tecnicos' definida.


In [9]:
# Verifica se os DataFrames processados existem antes de calcular
if 'diarios_proc' in locals():
    # Os DataFrames com os cálculos finais
    df_diario_calculado = calcular_indicadores_tecnicos(diarios_proc)
    df_semanal_calculado = calcular_indicadores_tecnicos(semanais_proc)
    df_mensal_calculado = calcular_indicadores_tecnicos(mensais_proc)

    print("Indicadores calculados para os 3 DataFrames (em clones separados).")
else:
    print("Os DataFrames processados ('diarios_proc', etc.) não foram criados. Verifique a Célula 4.")

try:
    if 'df_diario_calculado' in locals():
        # Define os nomes das novas tabelas
        tabela_diario_final = "diario_com_indicadores"
        tabela_semanal_final = "semanal_com_indicadores"
        tabela_mensal_final = "mensal_com_indicadores"

        # Salva os DataFrames no banco de dados
        # if_exists='replace': Substitui a tabela se ela já existir de uma execução anterior
        # index=False: Não salva o índice do Pandas como uma coluna no DB
        
        df_diario_calculado.to_sql(tabela_diario_final, con=engine, if_exists='replace', index=False)
        print(f"DataFrame diário salvo na tabela: '{tabela_diario_final}'")
        
        df_semanal_calculado.to_sql(tabela_semanal_final, con=engine, if_exists='replace', index=False)
        print(f"DataFrame semanal salvo na tabela: '{tabela_semanal_final}'")
        
        df_mensal_calculado.to_sql(tabela_mensal_final, con=engine, if_exists='replace', index=False)
        print(f"DataFrame mensal salvo na tabela: '{tabela_mensal_final}'")
        
        print("\nSucesso! Os dados calculados foram salvos em novas tabelas no arquivo 'dados_acoes.db'.")
        
    else:
        print("Os DataFrames calculados não foram encontrados. Pulei a etapa de salvar.")
        
except Exception as e:
    print(f"Erro ao salvar os dados no banco de dados: {e}")

Indicadores calculados para os 3 DataFrames (em clones separados).
DataFrame diário salvo na tabela: 'diario_com_indicadores'
DataFrame semanal salvo na tabela: 'semanal_com_indicadores'
DataFrame mensal salvo na tabela: 'mensal_com_indicadores'

Sucesso! Os dados calculados foram salvos em novas tabelas no arquivo 'dados_acoes.db'.
